In [26]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split

column aralarında ";" var çıkart

In [7]:
df=pd.read_csv("/content/TurkishSMSCollection.csv",on_bad_lines='skip',delimiter=';')
df.head(8)

,Message,Group,GroupText
0,125 lira,2,Normal
1,Baskanin aksam toplantısi fenaymis :),2,Normal
2,Bilal yalçnlara ne zaman gidiyoruz?,2,Normal
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",1,Spam
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,1,Spam
5,İyi ya dokuz eylül iyidir arş.gör falan kovala ;),2,Normal
6,İyiyim teşekkürler oturuyoruz nazarda arkadaşl...,2,Normal
7,Kapatamıyorun ayarlara girmem lazım :),2,Normal


grup ifadelerinin spam ya da normal mesaj olup olmadığı 1 ve 2 olarak sınıflandırılmıştı ama biz bunu 1 ve 0 olarak (binary) olarak değiştirdik

In [18]:
df['Label']=np.where(df['Group']==1,1,0)
df.head()

,Message,Group,GroupText,Label
0,125 lira,2,Normal,0
1,Baskanin aksam toplantısi fenaymis :),2,Normal,0
2,Bilal yalçnlara ne zaman gidiyoruz?,2,Normal,0
3,"BiP ile mesajlarimi aninda, daha eglenceli gon...",1,Spam,1
4,DIGITURKTEN FIRSAT! SiZE OZEL YIL SONUNA KADAR...,1,Spam,1


metni vektöre çevir

In [12]:
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(df['Message'])
sequences = tokenizer.texts_to_sequences(df['Message'])

In [19]:
pad=pad_sequences(sequences, maxlen=20)
y=df['Label'].values

In [23]:
sequences

[[],
 [],
 [103, 209],
 [13, 123, 69, 98, 40, 30, 8, 243],
 [77, 45, 9, 424, 17, 290, 33, 385, 67, 107, 148, 184, 35, 49, 3, 30, 189],
 [304, 90, 386],
 [98, 229, 40],
 [],
 [],
 [270, 160, 88],
 [305, 230, 74],
 [94],
 [94, 178, 88, 51, 178],
 [94],
 [94, 160],
 [94],
 [],
 [62, 2, 2, 24, 24, 6, 495, 10, 5, 37, 271, 11, 154, 4],
 [62, 351, 496, 122, 248, 83, 496, 248, 199, 2, 154, 218, 37, 352, 271, 12],
 [62, 312, 128, 6, 249, 260, 142, 151, 326, 5, 132, 1, 26, 12, 138],
 [64,
  158,
  457,
  63,
  361,
  64,
  6,
  136,
  479,
  479,
  4,
  5,
  25,
  29,
  479,
  7,
  219,
  19,
  11],
 [53,
  6,
  199,
  353,
  236,
  22,
  3,
  8,
  43,
  44,
  353,
  236,
  35,
  22,
  3,
  8,
  43,
  44,
  244,
  9,
  61,
  5,
  75,
  1,
  497,
  26,
  113,
  1,
  113,
  7],
 [24, 24, 6, 250, 17, 32, 2, 5, 37, 1, 271, 12, 11, 154, 4],
 [24, 24, 6, 23, 109, 32, 2, 185, 70, 3, 5, 37, 271, 154, 4],
 [24, 24, 6, 23, 109, 32, 2, 480, 2, 154, 4, 5, 37, 271],
 [24, 24, 6, 32, 2, 100, 17, 32, 2, 154, 2

In [24]:
pad[4]

array([  0,   0,   0,  77,  45,   9, 424,  17, 290,  33, 385,  67, 107,
       148, 184,  35,  49,   3,  30, 189], dtype=int32)

In [25]:
pad[23]

array([  0,   0,   0,   0,   0,  24,  24,   6,  23, 109,  32,   2, 185,
        70,   3,   5,  37, 271, 154,   4], dtype=int32)

In [27]:
x_train,x_test,y_train,y_test=train_test_split(pad,y,test_size=0.2)

In [31]:
model=Sequential()
model.add(Embedding(500, 100, input_length=20))
model.add(LSTM(64,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Epoch 1/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9935 - loss: 0.0147 - val_accuracy: 0.9916 - val_loss: 0.0228
Epoch 2/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9975 - loss: 0.0081 - val_accuracy: 0.9916 - val_loss: 0.0214
Epoch 3/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9991 - loss: 0.0040 - val_accuracy: 0.9926 - val_loss: 0.0206
Epoch 4/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9998 - loss: 0.0034 - val_accuracy: 0.9905 - val_loss: 0.0218
Epoch 5/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 0.9998 - loss: 0.0025 - val_accuracy: 0.9916 - val_loss: 0.0243


In [37]:
tahmin=" ALLAH ASKINA SU PROJEYI HALLET"

tahmin_array=tokenizer.texts_to_sequences([tahmin])
tahmin_array=pad_sequences(tahmin_array, maxlen=20)

model.predict(tahmin_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


array([[0.00028022]], dtype=float32)

In [39]:
model.save('smsModeliEgitilmis.h5')